<a href="https://colab.research.google.com/github/brokoli777/Emotion-Recogntion/blob/main/Retry1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import kagglehub

path = kagglehub.dataset_download("msambare/fer2013")

print("Path to dataset files:", path)

100%|██████████| 60.3M/60.3M [00:04<00:00, 15.7MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/msambare/fer2013/versions/1


In [4]:
import shutil

# Define source and destination paths
source = "/root/.cache/kagglehub/datasets/msambare/fer2013/versions/1"
destination = "/content/dataset"

# Move files to destination folder
shutil.move(source, destination)

print(f"Files moved to {destination}")

Files moved to /content/dataset


In [20]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import random

import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from keras.preprocessing.image import load_img
from keras.models import Sequential
from keras.layers import Input, Dense, Conv2D, Dropout, Flatten, MaxPooling2D, BatchNormalization

In [6]:
TRAIN_DIR = './dataset/test'
TEST_DIR = './dataset/train'

In [7]:
print(os.listdir(TRAIN_DIR))
print(os.listdir(TEST_DIR))

['disgust', 'surprise', 'sad', 'fear', 'angry', 'neutral', 'happy']
['disgust', 'surprise', 'sad', 'fear', 'angry', 'neutral', 'happy']


In [8]:
print(len(os.listdir(TRAIN_DIR+'/angry')))
print(len(os.listdir(TRAIN_DIR+'/disgust')))

958
111


In [9]:
def load_dataset(directory):
    image_paths = []
    labels = []

    for label in os.listdir(directory):
        label_path = os.path.join(directory, label)  # Use os.path.join
        if not os.path.isdir(label_path):  # Skip if it's not a directory
            continue

        for filename in os.listdir(label_path):  # Correctly use label_path
            image_path = os.path.join(label_path, filename)
            image_paths.append(image_path)
            labels.append(label)

        print(label, "Completed")

    return image_paths, labels

In [10]:
## convert into dataframe
train = pd.DataFrame()
train['image'], train['label'] = load_dataset(TRAIN_DIR)
# shuffle the dataset
train = train.sample(frac=1).reset_index(drop=True)
train.head()

disgust Completed
surprise Completed
sad Completed
fear Completed
angry Completed
neutral Completed
happy Completed


,image,label
0,./dataset/test/angry/PrivateTest_29669680.jpg,angry
1,./dataset/test/neutral/PublicTest_3762401.jpg,neutral
2,./dataset/test/happy/PublicTest_87222808.jpg,happy
3,./dataset/test/sad/PublicTest_31246071.jpg,sad
4,./dataset/test/sad/PrivateTest_91999709.jpg,sad


In [11]:
test = pd.DataFrame()
test['image'], test['label'] = load_dataset(TEST_DIR)
test.head()

disgust Completed
surprise Completed
sad Completed
fear Completed
angry Completed
neutral Completed
happy Completed


,image,label
0,./dataset/train/disgust/Training_40778169.jpg,disgust
1,./dataset/train/disgust/Training_63957495.jpg,disgust
2,./dataset/train/disgust/Training_94043084.jpg,disgust
3,./dataset/train/disgust/Training_62283849.jpg,disgust
4,./dataset/train/disgust/Training_64347942.jpg,disgust


In [12]:
def get_features(images):
    features = []
    for image in images:
        img = load_img(image, color_mode='grayscale')
        img = np.array(img)
        features.append(img)
    features = np.array(features)
    features = features.reshape(len(features), 48, 48, 1)
    return features

In [13]:
train_features = get_features(train['image'])
test_features = get_features(test['image'])

In [14]:
## normalize the image
x_train = train_features/255.0
x_test = test_features/255.0

In [15]:
## convert label to integer
from sklearn.preprocessing import LabelEncoder
encode = LabelEncoder()
encode.fit(train['label'])
y_train = encode.transform(train['label'])
y_test = encode.transform(test['label'])

In [16]:
y_train = to_categorical(y_train, num_classes=7)
y_test = to_categorical(y_test, num_classes=7)

In [17]:
input_shape = (48, 48, 1)
output_class = 7

In [21]:
#Different model
model = Sequential()

model.add(Input(input_shape))

# First convolutional block
model.add(Conv2D(64, kernel_size=(3, 3), activation='relu', input_shape=(48, 48, 1), padding='same'))
model.add(BatchNormalization())
model.add(Conv2D(64, kernel_size=(3, 3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.3))

# Second convolutional block
model.add(Conv2D(128, kernel_size=(3, 3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(Conv2D(128, kernel_size=(3, 3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.3))

# Third convolutional block
model.add(Conv2D(256, kernel_size=(3, 3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(Conv2D(256, kernel_size=(3, 3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.3))

# Flatten and fully connected layers
model.add(Flatten())
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(7, activation='softmax'))

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [22]:
model_info = model.fit(
        x=x_train,
        y=y_train,
        batch_size=64,
        epochs=100,
        verbose=1,
        validation_data=(x_test, y_test))

Epoch 1/100
113/113 ━━━━━━━━━━━━━━━━━━━━ 34s 159ms/step - accuracy: 0.2035 - loss: 7.7341 - val_accuracy: 0.1682 - val_loss: 2.1757
Epoch 2/100
113/113 ━━━━━━━━━━━━━━━━━━━━ 19s 68ms/step - accuracy: 0.2704 - loss: 1.7959 - val_accuracy: 0.2481 - val_loss: 2.4418
Epoch 3/100
113/113 ━━━━━━━━━━━━━━━━━━━━ 10s 66ms/step - accuracy: 0.3042 - loss: 1.7393 - val_accuracy: 0.1968 - val_loss: 3.6403
Epoch 4/100
113/113 ━━━━━━━━━━━━━━━━━━━━ 8s 67ms/step - accuracy: 0.3348 - loss: 1.6903 - val_accuracy: 0.3011 - val_loss: 1.8420
Epoch 5/100
113/113 ━━━━━━━━━━━━━━━━━━━━ 10s 68ms/step - accuracy: 0.3582 - loss: 1.6225 - val_accuracy: 0.3337 - val_loss: 1.7866
Epoch 6/100
113/113 ━━━━━━━━━━━━━━━━━━━━ 11s 79ms/step - accuracy: 0.3601 - loss: 1.6045 - val_accuracy: 0.3298 - val_loss: 1.9775
Epoch 7/100
113/113 ━━━━━━━━━━━━━━━━━━━━ 9s 67ms/step - accuracy: 0.4029 - loss: 1.5381 - val_accuracy: 0.3612 - val_loss: 1.6577
Epoch 8/100
113/113 ━━━━━━━━━━━━━━━━━━━━ 8s 67ms/step - accuracy: 0.4189 - loss: 1.5

In [23]:
model.evaluate(x_test, y_test, verbose=1)

898/898 ━━━━━━━━━━━━━━━━━━━━ 7s 7ms/step - accuracy: 0.5132 - loss: 3.4920


[2.9620962142944336, 0.5574907064437866]

In [24]:
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)



ValueError: The filename must end in `.weights.h5`. Received: filepath=model.h5

In [25]:
# save trained model weight in .h5 file
model.save_weights('model.weights.h5')

In [26]:
model.save("emotionmodel.keras")